In [218]:
from datetime import datetime
import json
from bs4 import BeautifulSoup
import requests
from dataclasses import dataclass, field
import types
from pprint import pprint

class Holiday:
    def __init__(self,name, date):
        self.name = name
        # only accept datetime objects as date
        self.date = datetime.strptime(date, "%Y-%m-%d")
    
    def __str__(self):
        return f"{self.name} ({self.date})"

In [219]:
class HolidayList:
    "HolidayList class"
    def __init__(self):
        self.innerHolidays = []

    def addHoliday(self, holidayObj):
        if isinstance(holidayObj, Holiday):
            # Make sure holidayObj is an Holiday Object by checking the type. Add to innerHolidays
            self.innerHolidays.append(holidayObj)
            print(f"Added holiday {holidayObj}")

    def findHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            # Find and return Holiday in innerHolidays
            if holiday.name == HolidayName and holiday.date == Date:
                return holiday
            else:
                print(f"{holiday} is not in list.")

    def removeHoliday(self, HolidayName, Date):
        for holiday in self.innerHolidays:
            # Find Holiday in innerHolidays by searching the name and date combination.
            if holiday.name == HolidayName and holiday.date == Date:
                # remove the Holiday from innerHolidays, and inform user you deleted the holiday
                self.innerHolidays.remove(holiday)
                print(f"{holiday} was removed from the list.")
            else:
                print(f"{holiday} is not in list.")

    def read_json(self, filelocation):
        # Read in things from json file location and add holidays to innerHolidays
        with open(filelocation, "r") as jsonfile:
            data = json.load(jsonfile)
            for i in range(len(data["holidays"])):
                holiday = Holiday(data["holidays"][i]["name"], data["holidays"][i]["date"])
                HolidayList.addHoliday(self, holiday)

    def save_to_json(self, filelocation):
        json_list = []
        json_dict = {"holidays": json_list}
        # Write out json file to selected file.
        with open(filelocation, "w") as jsonfile:
            for holiday in self.innerHolidays:
                json_list.append(holiday.__dict__)
            json.dump(json_dict, jsonfile, indent=3)

    def scrapeHolidays(self):
        def getHTML(url):
            response = requests.get(url)
            return response.text
        # Scrape holidays from 2020-2024
        for year in range(2020,2025):
            year = str(year)
            html = getHTML("https://www.timeanddate.com/calendar/custom.html?year=" + year +"&country=1&cols=3&df=1&hol=33554809")
            soup = BeautifulSoup(html,'html.parser')
            holiday_data = soup.find('table', attrs = {'class':'cht lpad'}).find('tbody')
            for row in holiday_data:
                items = row.find_all('tr')
                holiday_dict = {}
                holiday_dict['name'] = row.find('td').find_next('td').string
                holiday_dict['date'] = f"{row.find('td').string}, {year}"
                # Check to see if name and date of holiday is in innerHolidays array. If it is, don't add it to list
                if holiday_dict['name'] in self.innerHolidays and holiday_dict['date'] in self.innerHolidays:
                    continue
                else:
                    self.innerHolidays.append(holiday_dict)
    
    def numHolidays(self):
        # Return the total number of holidays in innerHolidays
        return len(self.innerHolidays)

    def filter_holidays_by_week(self, year, week_number):
        # Use a Lambda function to filter by week number and cast results as list
        global filter_by_week
        filter_by_week = filter(lambda holiday:datetime.strftime(holiday.date, "%U") == week_number
        and datetime.strftime(holiday.date, "%Y") == year, self.innerHolidays)
        filter_by_week = list(filter_by_week)
        return(filter_by_week)
    
    def displayHolidaysInWeek(self, year, week_number):
        # Print list of holidays within a week as a parameter
        HolidayList.filter_holidays_by_week(self, year, week_number)
        for row in range(len(filter_by_week)):
            print(filter_by_week[row])

In [220]:
holiday1 = Holiday("New Year's Day", "2022-01-01")
print(holiday1)

holidaylist = HolidayList()

holidaylist.addHoliday(holiday1)
#holidaylist.findHoliday("New Year's Day", "2022-01-01")
#holidaylist.removeHoliday("New Year's Day", "2022-01-01")

#holidaylist.read_json("holidays.json")

#holidaylist.scrapeHolidays()

#holidaylist.numHolidays()
holidaylist.displayHolidaysInWeek("2022", "00")



New Year's Day (2022-01-01 00:00:00)
Added holiday New Year's Day (2022-01-01 00:00:00)
New Year's Day (2022-01-01 00:00:00)


In [221]:
import requests

url = "https://community-open-weather-map.p.rapidapi.com/forecast/daily"

querystring = {"q":"new york,us","lat":"41","lon":"74","cnt":"7","units":"imperial"}

headers = {
    'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
    'x-rapidapi-key': "1e19830f3bmshf56abb61a6e9168p189818jsnf035a12afc26"
    }

weather = requests.request("GET", url, headers=headers, params=querystring)
weather_dict = weather.json()

print(weather_dict)

print(weather_dict["list"][0]["weather"][0]["main"])

{'city': {'id': 5128581, 'name': 'New York', 'coord': {'lon': -74.006, 'lat': 40.7143}, 'country': 'US', 'population': 8175133, 'timezone': -18000}, 'cod': '200', 'message': 17.7835028, 'cnt': 7, 'list': [{'dt': 1641747600, 'sunrise': 1641730779, 'sunset': 1641764775, 'temp': {'day': 37.26, 'min': 28.51, 'max': 39.51, 'night': 39.51, 'eve': 38.1, 'morn': 31.48}, 'feels_like': {'day': 27.77, 'night': 31.73, 'eve': 31.15, 'morn': 22.12}, 'pressure': 1021, 'humidity': 75, 'weather': [{'id': 501, 'main': 'Rain', 'description': 'moderate rain', 'icon': '10d'}], 'speed': 16.8, 'deg': 219, 'gust': 36.95, 'clouds': 99, 'pop': 0.96, 'rain': 2.62}, {'dt': 1641834000, 'sunrise': 1641817166, 'sunset': 1641851236, 'temp': {'day': 28.22, 'min': 24.96, 'max': 37.8, 'night': 24.96, 'eve': 28.08, 'morn': 27.79}, 'feels_like': {'day': 16.72, 'night': 12.52, 'eve': 17.53, 'morn': 16.84}, 'pressure': 1022, 'humidity': 42, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'